In [1]:
# https://gist.github.com/alexweberk/635431b5c5773efd6d1755801020429f

from mlx_lm import generate, load

model, tokenizer = load("google/gemma-3-4b-it")

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

In [2]:
messages = [{"role": "user", "content": "Why is the sky blue?"}]
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

'<bos><start_of_turn>user\nWhy is the sky blue?<end_of_turn>\n<start_of_turn>model\n'

In [3]:
# using sampler to control generation parameters
from mlx_lm.sample_utils import make_sampler    
sampler = make_sampler(temp=0.0, top_p=1.0)


# Generating without adding a prompt template manually
prompt = """
Why is the sky blue?
""".strip()
response = generate(
    model,
    tokenizer,
    prompt=prompt,
    sampler = sampler,
    verbose=True,  # Set to True to see the prompt and response
    max_tokens=256,
)



The sky appears blue due to a phenomenon called Rayleigh scattering. Here's a breakdown of how it works:

1. **Sunlight and Colors:** Sunlight is actually made up of all the colors of the rainbow. You can see this when sunlight passes through a prism, separating into its constituent colors.

2. **Entering the Atmosphere:** When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).

3. **Scattering of Light:** This collision causes the sunlight to scatter in different directions.

4. **Rayleigh Scattering:**  Rayleigh scattering is the key. It states that shorter wavelengths of light (blue and violet) are scattered *much* more strongly than longer wavelengths (red and orange).  The amount of scattering is inversely proportional to the fourth power of the wavelength.  This means blue light (shorter wavelength) is scattered about 10 times more than red light.

5. **Why Blue, Not Violet?** Violet light is scattered even *more* than blue

In [4]:
from datasets import load_dataset

dataset = load_dataset("teknium/trismegistus-project")
dataset

DatasetDict({
    train: Dataset({
        features: ['topic', 'domain_task_type', 'id', 'system_prompt_used', 'source', 'conversations'],
        num_rows: 13528
    })
})

In [5]:
# convert the dataset to pandas dataframe
import pandas as pd

df = pd.DataFrame(dataset["train"])
df.head()

,topic,domain_task_type,id,system_prompt_used,source,conversations
0,'Big Man' society,Task,570a8404-3270-4aba-a47c-660359440835,"You are a master of the esoteric, occult, 'Big...",DomainExpert_Occult,"[{'from': 'human', 'value': 'Compose a compreh..."
1,'Big Man' society,Task,ddf44765-8756-46db-a945-672050905fc0,"You are a master of the esoteric, occult, 'Big...",DomainExpert_Occult,"[{'from': 'human', 'value': 'Develop an intric..."
2,'Big Man' society,Task,9ef38c3a-31ed-48d7-94d2-75fc588bcb2e,"You are a master of the esoteric, occult, 'Big...",DomainExpert_Occult,"[{'from': 'human', 'value': 'Write an extensiv..."
3,'Big Man' society,Task,6dea7781-0f74-4692-8d1d-762c6585c280,"You are a master of the esoteric, occult, 'Big...",DomainExpert_Occult,"[{'from': 'human', 'value': 'Develop an intric..."
4,'Black Books' of European necromancy,Task,188c6c15-d2b7-448b-b93c-505aeca2a458,"You are a master of the esoteric, occult, 'Bla...",DomainExpert_Occult,"[{'from': 'human', 'value': 'Devise an intrica..."


In [6]:
print(df.iloc[0]["system_prompt_used"], "\n", df.iloc[0]["conversations"])

You are a master of the esoteric, occult, 'Big Man' society, you complete tasks to the absolute best of your ability, no matter if you think you were not trained to do the task specifically, you will attempt to do it anyways, since you have performed the tasks you are given with great mastery, accuracy, and deep understanding of what is requested. You do the tasks faithfully, and stay true to the mode and domain's mastery role. If the task is not specific enough, note that and create specifics that enable completing the task. 
 [{'from': 'human', 'value': "Compose a comprehensive biography of a renowned figure in the 'Big Man' society, delving into their connections to the occult and esoteric realms. Emphasize their influence on the contemporary 'Big Man' society, their mystical practices, and the transmission of arcane knowledge. Ensure to explore the convergence of their occult work with the politics and power dynamics of the 'Big Man' establishment, dissecting how it fueled their ri

In [7]:
# Split the quetion and answer into separate columns
df[["question", "answer"]] = pd.DataFrame(df["conversations"].tolist(), index=df.index)

# Only keep the 'value' portion of the JSON
df["question"] = df["question"].apply(lambda x: x["value"])
df["answer"] = df["answer"].apply(lambda x: x["value"])

df[["system_prompt_used", "question", "answer"]]

,system_prompt_used,question,answer
0,"You are a master of the esoteric, occult, 'Big...",Compose a comprehensive biography of a renowne...,Title: The Mystifying Odyssey of Eliphas Black...
1,"You are a master of the esoteric, occult, 'Big...",Develop an intricate numerology system that de...,I. Foundational Numerology\n\nThe 'Big Man' so...
2,"You are a master of the esoteric, occult, 'Big...",Write an extensive biography of a prominent oc...,Title: Nathaniel Ziester: A Life in Shadows - ...
3,"You are a master of the esoteric, occult, 'Big...","Develop an intricate system of numerology, inc...",Title: The Numerological Riddles of the Big Ma...
4,"You are a master of the esoteric, occult, 'Bla...",Devise an intricate multi-step process for the...,Step 1: Assess the condition of the grimoire\n...
...,...,...,...
13523,"You are a master of the esoteric, occult, Reap...",In the context of the Reappropriated Goddess m...,Answer: To regain women's empowerment and infl...
13524,"You are a master of the esoteric, occult, Reap...",Write a section of a grimoire explaining the c...,Title: The Reappropriated Goddess in the Occul...
13525,"You are a master of the esoteric, occult, Reap...",Write a section of a grimoire specifically foc...,Title: The Reappropriated Goddess: A Journey i...
13526,"You are a master of the esoteric, occult, Reap...",Create a detailed introductory section for a g...,Title: The Reappropriated Goddess: A Grimoire ...


In [8]:
def generate_prompt(row: pd.Series) -> str:
    "Format to Gemma's chat template"
    return """<bos><start_of_turn>user
## Instructions
{}
## User
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(row["system_prompt_used"], row["question"], row["answer"])


df["text"] = df.apply(generate_prompt, axis=1)

# Let's see what the model will be trained on
print(df["text"].iloc[0])

<bos><start_of_turn>user
## Instructions
You are a master of the esoteric, occult, 'Big Man' society, you complete tasks to the absolute best of your ability, no matter if you think you were not trained to do the task specifically, you will attempt to do it anyways, since you have performed the tasks you are given with great mastery, accuracy, and deep understanding of what is requested. You do the tasks faithfully, and stay true to the mode and domain's mastery role. If the task is not specific enough, note that and create specifics that enable completing the task.
## User
Compose a comprehensive biography of a renowned figure in the 'Big Man' society, delving into their connections to the occult and esoteric realms. Emphasize their influence on the contemporary 'Big Man' society, their mystical practices, and the transmission of arcane knowledge. Ensure to explore the convergence of their occult work with the politics and power dynamics of the 'Big Man' establishment, dissecting how 

In [9]:
from pathlib import Path

Path("data").mkdir(exist_ok=True)

split_ix = int(len(df) * 0.9)
# shuffle data
data = df.sample(frac=1, random_state=42)
train, valid = data[:split_ix], data[split_ix:]

# Save train and valid dataset as jsonl files
train[["text"]].to_json("data/train.jsonl", orient="records", lines=True, force_ascii=False)
valid[["text"]].to_json("data/valid.jsonl", orient="records", lines=True, force_ascii=False)

!head -n 1 data/train.jsonl

{"text":"<bos><start_of_turn>user\n## Instructions\nYou are a master of the esoteric, occult, Chronotopic inversion and education, you have written many textbooks on the subject in ways that provide students with rich and deep understanding of the subject. You are being asked to write textbook-like sections on a topic and you do it with full context, explainability, and reliability in accuracy to the true facts of the topic at hand, in a textbook style that a student would easily be able to learn from, in a rich, engaging, and contextual way. Always include relevant context (such as formulas and history), related concepts, and in a way that someone can gain deep insights from.\n## User\nWrite a detailed explanation of Chronotopic inversion within the context of the occult, focusing on its history, methodology, practical applications, and key concepts. Elaborate on how an adept in the esoteric arts can harness this mysterious power to manipulate time and space for personal growth, spiri

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
!python -m mlx_lm.lora --help

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Calling `python -m mlx_lm.lora...` directly is deprecated. Use `mlx_lm.lora...` or `python -m mlx_lm lora ...` instead.
usage: lora.py [-h] [--model MODEL] [--train] [--data DATA]
               [--fine-tune-type {lora,dora,full}]
               [--optimizer {adam,adamw,sgd,adafactor}] [--mask-prompt]
               [--num-layers NUM_LAYERS] [--batch-size BATCH_SIZE]
               [--iters ITERS] [--val-batches VAL_BATCHES]
               [--learning-rate LEARNING_RATE]
               [--steps-per-report STEPS_PER_REPORT]
               [--steps-per-eval STEPS_PER_EVAL]
               [--resume-adapter-file RESUME_ADAPTER_FILE]
               [--adapter-path ADAPTER_PATH] [--save-every SAVE_EVERY]
               [--test] [--test-batches TEST_BATCHES]
               [--max-seq-length MAX_SEQ_LENGTH] [-c CONFIG]
               [--grad-checkpoint] [--wandb WANDB] [--seed SEED]

LoRA or QLoRA finetuning.

options:
  -h, --help            show this help message and exit
  --model MODEL    

In [ ]:
!python -m mlx_lm lora \
    --model google/gemma-3-4b-it \
    --train \
    --iters 600 \
    --data data \
    --lora-layers 4
    # --resume-adapter-file checkpoints/600_adapters.npz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: __main__.py [-h] [--model MODEL] [--train] [--data DATA]
                   [--fine-tune-type {lora,dora,full}]
                   [--optimizer {adam,adamw,sgd,adafactor}] [--mask-prompt]
                   [--num-layers NUM_LAYERS] [--batch-size BATCH_SIZE]
                   [--iters ITERS] [--val-batches VAL_BATCHES]
                   [--learning-rate LEARNING_RATE]
                   [--steps-per-report STEPS_PER_REPORT]
                   [--steps-per-eval STEPS_PER_EVAL]
                   [--resume-adapter-file RESUME_ADAPTER_FILE]
                   [--adapter-path ADAPTER_PATH] [--save-every SAVE_EVERY]
                   [--test] [--test-batches TEST_BATCHES]
                   [--max-seq-length MAX_SEQ_LENGTH] [-c CONFIG]
                   [--grad-checkpoint] [--wandb WANDB] [--seed SEED]
__main__.py: error: unrecognized arguments: --lora-layers 4


In [15]:
!python -m mlx_lm.generate --model "google/gemma-3-4b-it" \
               --adapter-file checkpoints/600_adapters.npz \
               --max-tokens 256 \
               --prompt "Why is the sky blue?" \
               --seed 69


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<frozen runpy>:128: RuntimeWarning: 'mlx_lm.generate' found in sys.modules after import of package 'mlx_lm', but prior to execution of 'mlx_lm.generate'; this may result in unpredictable behaviour
Calling `python -m mlx_lm.generate...` directly is deprecated. Use `mlx_lm.generate...` or `python -m mlx_lm generate ...` instead.
usage: generate.py [-h] [--model MODEL] [--adapter-path ADAPTER_PATH]
                   [--extra-eos-token EXTRA_EOS_TOKEN [EXTRA_EOS_TOKEN ...]]
                   [--system-prompt SYSTEM_PROMPT] [--prompt PROMPT]
                   [--prefill-response PREFILL_RESPONSE]
                   [--max-tokens MAX_TOKENS] [--temp TEMP] [--top-p TOP_P]
                   [--min-p MIN_P] [--top-k TOP_K]
                   [--xtc-probability XTC_PROBABILITY]
                   [--xtc-threshold XTC_THRESHOLD]
                   [--min-tokens-to-keep MIN_TOKENS_TO_KEEP] [--seed SEED]
                   [--ignore-chat-template] [--use-default-chat-template]
                 

In [16]:
# I thought this system prompt was cool, so let's use this one
system_prompt = df["system_prompt_used"].unique()[-2]
# system_prompt = "You are a master in the field of the esoteric, occult, Reappropriated Goddess and Education. You are a writer of tests, challenges, books and deep knowledge on Reappropriated Goddess for initiates and students to gain deep insights and understanding from. You write answers to questions posed in long, explanatory ways and always explain the full context of your answer (i.e., related concepts, formulas, examples, or history), as well as the step-by-step thinking process you take to answer the challenges. Be rigorous and thorough, and summarize the key themes, ideas, and conclusions at the end."
print(system_prompt)

You are a master in the field of the esoteric, occult, Reappropriated Goddess and Education. You are a writer of tests, challenges, books and deep knowledge on Reappropriated Goddess for initiates and students to gain deep insights and understanding from. You write answers to questions posed in long, explanatory ways and always explain the full context of your answer (i.e., related concepts, formulas, examples, or history), as well as the step-by-step thinking process you take to answer the challenges. Be rigorous and thorough, and summarize the key themes, ideas, and conclusions at the end.


In [24]:
!python -m mlx_lm.generate --model "google/gemma-3-4b-it" \
               --adapter-path "checkpoints/600_adapters.npz" \
               --max-tokens 256 \
               --prompt "Why is the sky blue?" \
               --seed 69

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<frozen runpy>:128: RuntimeWarning: 'mlx_lm.generate' found in sys.modules after import of package 'mlx_lm', but prior to execution of 'mlx_lm.generate'; this may result in unpredictable behaviour
Calling `python -m mlx_lm.generate...` directly is deprecated. Use `mlx_lm.generate...` or `python -m mlx_lm generate ...` instead.
Fetching 13 files: 100%|█████████████████████| 13/13 [00:00<00:00, 74693.08it/s]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/yoonani/Development/prac_llm/.venv/lib/python3.11/site-packages/mlx_lm/generate.py", line 908, in <module>
    main()
  File "/Users/yoonani/Development/prac_llm/.venv/lib/python3.11/site-packages/mlx_lm/generate.py", line 815, in main
    model, tokenizer = load(
                       ^^^^^
  File "/Users/yoonani/Development/prac_llm/.venv/lib/python3.11/site-packages/mlx_lm/utils.py", line 262, in load
    model = load_adapters(

In [17]:
question = "Why is the sky blue?"


def format_prompt(system_prompt: str, question: str) -> str:
    "Format the question to the format of the dataset we fine-tuned to."
    return """<bos><start_of_turn>user
## Instructions
{}
## User
{}<end_of_turn>
<start_of_turn>model
""".format(
        system_prompt, question
    )


print(format_prompt(system_prompt, question))

<bos><start_of_turn>user
## Instructions
You are a master in the field of the esoteric, occult, Reappropriated Goddess and Education. You are a writer of tests, challenges, books and deep knowledge on Reappropriated Goddess for initiates and students to gain deep insights and understanding from. You write answers to questions posed in long, explanatory ways and always explain the full context of your answer (i.e., related concepts, formulas, examples, or history), as well as the step-by-step thinking process you take to answer the challenges. Be rigorous and thorough, and summarize the key themes, ideas, and conclusions at the end.
## User
Why is the sky blue?<end_of_turn>
<start_of_turn>model



In [21]:
# Load the fine-tuned model with LoRA weights
model_lora, _ = load(
    "google/gemma-3-4b-it",
    adapter_path="./adapters.npz",  # adapters.npz is the final checkpoint saved at the end of training
)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

FileNotFoundError: The adapter path does not exist: adapters.npz